### Setup

1. Create AI Search Connection
2. Update AI_SEARCH_CONNECTION_NAME
3. Update AI_SEARCH_INDEX_NAME

### Connect to Foundry project

In [ ]:
# Import packages
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(override=True) # Load environment variables from .env file

# Use the connection string to connect to your Foundry project
project_connection_string = os.getenv("AIPROJECT_CONNECTION_STRING")
try:
    project = AIProjectClient.from_connection_string(
        conn_str=project_connection_string, credential=DefaultAzureCredential()
    )
except Exception as e:
    print(f"Failed to connect to project: {e}")
    raise



In [ ]:
from azure.ai.projects.models import AzureAISearchTool, AzureAISearchQueryType

connection = project.connections.get(connection_name=os.environ["AI_SEARCH_CONNECTION_NAME"])
conn_id = connection.id
indexName = os.environ["AI_SEARCH_INDEX_NAME"]
print(conn_id)

# Initialize agent AI search tool and add the search index connection id
ai_search = AzureAISearchTool(
    index_connection_id=conn_id, index_name=indexName, query_type=AzureAISearchQueryType.SEMANTIC, top_k=3, filter=""
)

# Create agent with AI search tool and process assistant run

agent = project.agents.create_agent(
    model=os.getenv("CHAT_MODEL"),
    name="AISearchAgent",
    instructions="You are a helpful assistant",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources
)

### Create a thread for conversation presistance

In [ ]:
# Create a thread
thread = project.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

In [ ]:
def sendMessage(msg):
    # Get messages from the thread
    message = project.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=msg
        )
    print(f"Created message, ID: {message.id}")

    run = project.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            print(f"Run failed: {run.last_error}")
            raise Exception(f"Run failed: {run.last_error}")

    messages = project.agents.list_messages(thread_id=thread.id, order="asc", after=message.id)
    for msg in messages.data:
        role = msg.role
        content = msg.content[0].text.value
        print(f"{role.capitalize()}: {content}")


### Get the response

In [ ]:
sendMessage("Do you have his SEC fillinging?")